In [5]:
import json
import numpy as np
from transformers import AutoTokenizer

MODEL_ID = "/root/autodl-tmp/TISER/model/Qwen/Qwen2.5-7B-Instruct"  # 你的模型路径
DATA_PATH = "/root/autodl-tmp/TISER/data/TISER_formatted_train.jsonl" # 你的数据路径

print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

lengths = []
print("正在统计数据长度...")

# 2. 扫描数据
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        # 假设数据格式是 standard messages list
        if "messages" in item:
            # 使用 chat 模板将对话转成纯文本进行统计
            text = tokenizer.apply_chat_template(item["messages"], tokenize=False)
            # 计算 token 数量
            token_ids = tokenizer(text)["input_ids"]
            lengths.append(len(token_ids))

# 3. 分析结果
lengths = np.array(lengths)
print("\n" + "="*30)
print(f"数据总量: {len(lengths)} 条")
print(f"最短样本: {np.min(lengths)} tokens")
print(f"最长样本: {np.max(lengths)} tokens")
print(f"平均长度: {int(np.mean(lengths))} tokens")
print("-" * 30)
print(f"P50 (中位数): {int(np.percentile(lengths, 50))} tokens")
print(f"P90 (90%的数据小于): {int(np.percentile(lengths, 90))} tokens")
print(f"P95 (95%的数据小于): {int(np.percentile(lengths, 95))} tokens")
print(f"P99 (99%的数据小于): {int(np.percentile(lengths, 99))} tokens")
print("="*30)

# 4. 判断 2048 是否合适
over_limit = np.sum(lengths > 1024)
ratio = over_limit / len(lengths) * 100
print(f"\n超过 2048 长度的样本数: {over_limit} ({ratio:.2f}%)")

if ratio > 10:
    print("⚠️ 警告: 超过 10% 的数据会被截断，建议增加 max_seq_length 或筛选数据。")
else:
    print("✅ 安全: 绝大多数数据都在范围内，使用 2048 没问题。")

正在加载 Tokenizer...
正在统计数据长度...

数据总量: 54488 条
最短样本: 598 tokens
最长样本: 2021 tokens
平均长度: 887 tokens
------------------------------
P50 (中位数): 841 tokens
P90 (90%的数据小于): 1121 tokens
P95 (95%的数据小于): 1230 tokens
P99 (99%的数据小于): 1492 tokens

超过 2048 长度的样本数: 0 (0.00%)
✅ 安全: 绝大多数数据都在范围内，使用 2048 没问题。
